In [1]:
import PIL
from PIL import ImageFont
from PIL import Image 
from PIL import ImageDraw
from PIL import ImageFilter

import os
import cv2
from pascal_voc_writer import Writer
import shutil
import numpy as np

import random
from random import randrange

import numpy
import pandas as pd

from tqdm import tqdm

In [2]:
## Random PLate Maker ##

lets = 'ABCDEFGHIJKMNOPQRSTUVWXYZ'
num = '0123456789'

total_plates_to_make = 500 # for each type
plate_size = (110,54)

coord_list = [  [(5 ,18),(25,18),(47,18),(70,18) ,(94,18) ,(114,18),(134,18)],
                [(5,25),(25,25),(45,25),(100,25),(122,25),(142,25)],
                [(20,20),(40,20),(60,20),(90,20),(110,20),(130,20)],
                [(20,25),(40,25),(60,25),(90,25),(110,25),(130,25)],
                [(5 ,25),(25,25),(47,25),(70,25),(94 ,25),(114,25),(134,25)],
                [(5 ,25),(25,25),(45,25),(95,25),(115,25),(135,25)],
                [(20,20),(40,20),(60,20),(90,20),(110,20),(130,20)],
                [(5 ,25),(25,25),(45,25),(95,25),(115,25),(135,25)],
              
                [(5 ,18),(25,18),(47,18),(70,18) ,(94,18) ,(114,18),(134,18)],
                [(30 ,18),(50,18),(70,18),(90,18) ,(120,18),(140,18)],
                [(20,22),(40,22),(60,22),(100,22),(120,22),(140,22)],
                [(15,25),(35,25),(55,25),(95,25),(115,25),(135,25)],
                [(20,25),(40,25),(60,25),(100,25),(120,25),(140,25)],
                [(5 ,25),(25,25),(47,25),(85,25) ,(105,25) ,(125,25),(145,25)],
                [(15,25),(35,25),(55,25),(95,25),(115,25),(135,25)],         
                [(20,25),(40,25),(60,25),(100,25),(120,25),(140,25)],    
                [(15 ,18),(35,18),(55,18),(95,18) ,(115,18),(135,18)],
                [(15 ,18),(35,18),(55,18),(95,18) ,(115,18),(135,18)],   
                [(15 ,18),(35,18),(55,18),(95,18) ,(115,18),(135,18)],
                [(20 ,15),(40,15),(60,15),(90,15) ,(110,15),(130,15)],
                [(10 ,18),(30,18),(50,18),(90,18) ,(110,18) ,(130,18),(150,18)],
                [(15 ,18),(35,18),(55,18),(95,18) ,(115,18),(135,18)],
                [(10 ,25),(30,25),(50,25),(90,25) ,(110,25) ,(130,25),(150,25)],
                [(15 ,18),(35,18),(55,18),(95,18) ,(115,18),(135,18)],  
                [(20 ,18),(40,18),(60,18),(90,18) ,(110,18),(130,18)],      
                [(10 ,20),(30,20),(50,20),(80,20) ,(100,20) ,(120,20),(140,20)],
                [(10 ,20),(30,20),(50,20),(95,20),(115,20),(135,20)],
                [(10 ,20),(30,20),(50,20),(100,20),(120,20),(140,20)],
                [(10 ,20),(30,20),(50,20),(100,20),(120,20),(140,20)],
                [(10 ,20),(30,20),(50,20),(100,20),(120,20),(140,20)],
                [(5 ,18),(25,18),(47,18),(70,18) ,(94,18) ,(114,18),(134,18)]      
             ]


root_path   = 'images///' 
output_path = 'random_plates//'
os.makedirs(output_path,exist_ok=True)

for img_ind , coord in enumerate(tqdm(coord_list)):
        
    font = ImageFont.truetype("stuff//LICENSE PLATE USA.ttf",38)
    for i in range(total_plates_to_make):
        img=Image.open(root_path+str(img_ind+1)+'.jpg')
        draw = ImageDraw.Draw(img)
        inc = []
        if img_ind in [28,29,30]:
            coord = coord_list[random.choice(range(len(coord_list)-4))]
        for letter in range(len(coord)):
            coord_tuple = coord[letter]
            letter = random.choice(random.choice([num,lets]))
            inc.append(letter)
            if img_ind in [21,23]:
                draw.text(coord_tuple,letter,(255,255,255) if len(img.split()) > 1 else (255),font=font)
            else:
                draw.text(coord_tuple,letter,(0,0,0) if len(img.split()) > 1 else (0),font=font)

        img = img.resize(plate_size, Image.ANTIALIAS)
        img.save(output_path+str(''.join(inc))+'.jpg')
            
            
    


100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [01:02<00:00,  2.00s/it]


In [3]:
def find_coeffs(source_coords, target_coords):
    matrix = []
    for s, t in zip(source_coords, target_coords):
        matrix.append([t[0], t[1], 1, 0, 0, 0, -s[0]*t[0], -s[0]*t[1]])
        matrix.append([0, 0, 0, t[0], t[1], 1, -s[1]*t[0], -s[1]*t[1]])
    A = numpy.matrix(matrix, dtype=numpy.float)
    B = numpy.array(source_coords).reshape(8)
    res = numpy.dot(numpy.linalg.inv(A.T * A) * A.T, B)
    return numpy.array(res).reshape(8)

import csv
def WriteDictToCsv(name,dict_data,csv_header,header_flag,subdir = ''):
    try:
        if len(subdir)>0:
            os.makedirs(str(subdir)+"//",exist_ok=True)
            filename = subdir+"//"+name
        else:
            filename = name

        with open(filename+'.csv','a',newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_header)
            if header_flag:
                writer.writeheader()
            writer.writerow(dict_data)
    except IOError as error:
        errno,strerr,  = error.args
        print("I/O error({0}): {1}".format(errno, strerr))    
    return

In [10]:
# Prospective augment and paste randomly on backgrounds 

bg_path = './bgs/'
augmented_img_path = './random_plates/'
output_path = './plate_with_bgs/'
os.makedirs(output_path,exist_ok=True)

lbl_path = './plate_with_bgs_lbls/'
os.makedirs(lbl_path,exist_ok=True)

test_annot = False
image_resize = 320
# plate_size = (50,20)

augmented_img_path_lst = os.listdir(augmented_img_path)
random.shuffle(augmented_img_path_lst)


######### CSV ############
write_dict = {'img': None,'xmin' : None,'ymin' : None,'w': None,'h': None,'class': None}
csv_path = './train_csv//'
os.makedirs(csv_path , exist_ok=True)
csv_name =  csv_path+ 'Plates'
flag = True
if os.path.exists(csv_name+'csv'): flag = False

transform_per = 0
#########################

for i,x in enumerate(tqdm(augmented_img_path_lst)):
    if x.endswith('.jpg'):
        
        background = Image.open(bg_path+random.choice(os.listdir(bg_path))).convert("RGBA")
        img = Image.open(augmented_img_path+x).convert("RGBA")
            
        background = background.resize((image_resize,image_resize), Image.ANTIALIAS)
        w,h = background.size
        
        if i%2==0:
            top_left=(random.randint(0,30), random.randint(120,140))    # 0   , 140
            top_right=(random.randint(0+plate_size[0],30+plate_size[0]), random.randint(120,130))  # 120 , 100 
            bot_right=(random.randint(0+plate_size[0],  30+plate_size[0]), random.randint(120+plate_size[1],140+plate_size[1]))# 100 , 300
            bot_left=(random.randint(0,20), random.randint(120+plate_size[1],140+plate_size[1]))   # 10  , 250
        else:
            top_left=(random.randint(130,160), random.randint(100,120))
            top_right=(random.randint(130+plate_size[0],160+plate_size[0]), random.randint(100,110))
            bot_right=(random.randint(130+plate_size[0],160+plate_size[0]), random.randint(100+plate_size[1],120+plate_size[1]))
            bot_left=(random.randint(130,150), random.randint(100+plate_size[1],120+plate_size[1]))


        if i%3==0:
            top_left  = (0, 0)
            top_right = (110, 0)
            bot_right = (110, 54)
            bot_left  = (0, 54)
            coeffs = find_coeffs([(0, 0), (110, 0), (110, 54), (0, 54)],[top_left, top_right, bot_right, bot_left])
            img =img.transform((500, 500), Image.PERSPECTIVE, coeffs,Image.BICUBIC,fill=0)
        else:
            
            if (transform_per/len(augmented_img_path_lst)) < 0.90:
                transform_per+=1
            else:
                top_left  = (0, 0)
                top_right = (110, 0)
                bot_right = (110, 54)
                bot_left  = (0, 54)
                
            coeffs = find_coeffs([(0, 0), (110, 0), (110, 54), (0, 54)],[top_left, top_right, bot_right, bot_left])
            img =img.transform((500, 500), Image.PERSPECTIVE, coeffs,Image.BICUBIC,fill=0)

        box_x = random.randint(1,70)
        box_y = random.randint(1,100)
        
        rand_scale=random.randint(0,2)
        
        if True:#rand_scale in [0,1]:
            scale = random.uniform(0.3,0.4)
            img2 = img.resize((int(img.size[0] * scale),int(img.size[1] * scale)), resample=0)

            xmin = min(int(top_left[0] * scale +box_x), int(bot_left[0] * scale + box_x))
            ymin = min(int(top_left[1] * scale +box_y), int(top_right[1] * scale+ box_y))
            xmax = max(int(top_right[0] * scale +box_x), int(bot_right[0] * scale + box_x))
            ymax = max(int(bot_left[1] * scale +box_y), int(bot_right[1] * scale + box_y))
            xmax = w if xmax > w else xmax
            ymax = h if ymax > h else ymax

        else:

            xmin = min(int(top_left[0] +box_x), int(bot_left[0]  + box_x))
            ymin = min(int(top_left[1] +box_y), int(top_right[1] + box_y))
            xmax = max(int(top_right[0] +box_x), int(bot_right[0] + box_x))
            ymax = max(int(bot_left[1] +box_y), int(bot_right[1] + box_y))
            xmax = w if xmax > w else xmax
            ymax = h if ymax > h else ymax

            img2 = img.copy()
        background.paste(img2, (box_x,box_y), img2)
        background = background.convert("RGB")
        rand_scale_blr=random.randint(0,1)
        
        if rand_scale_blr==1:
            background = background.filter(ImageFilter.GaussianBlur(radius = 0.8))
        
        background.save(output_path+x.split('.')[0]+'_bg.jpg')
        
        writer = Writer(output_path+x.split('.')[0]+'_bg.jpg', w,h)
        writer.addObject('Number_Plate', xmin, ymin, xmax, ymax)
        writer.save(lbl_path+x.split('.')[0]+'_bg.xml')
        
        # writing CSV annotations         
        
        write_dict['img']  = x.split('.')[0]+'_bg'
        write_dict['xmin'] = int(xmin)
        write_dict['ymin'] = int(ymin)
        write_dict['w']    = int(xmax)
        write_dict['h']    = int(ymax)
        write_dict['class']= 0
        
        
        WriteDictToCsv(csv_name,write_dict,write_dict.keys(),flag)
        write_dict = dict.fromkeys(write_dict.keys(),None)
        flag = False
        
        

In [5]:
test_annot = False
if test_annot:
    import xml.etree.ElementTree as ET
    for xml_file in [f for f in os.listdir('./plate_with_bgs_lbls')[:5] if f.endswith(".xml")]:
        tree = ET.parse(os.path.join('./plate_with_bgs_lbls', xml_file))
        root = tree.getroot()

        file_name = None

        for elem in root:
            if elem.tag == 'filename':
                file_name = 'a'

        if elem.tag == 'object':
            obj_name = None
            coords = []
            for subelem in elem:
                if subelem.tag == 'name':
                    obj_name = subelem.text
                if subelem.tag == 'bndbox':
                    xmin1 = ""
                    xmax1 = ""
                    ymin1 = ""
                    ymax1 = ""
                    for subsubelem in subelem:
                        f = float(subsubelem.text)
                        i = int(f)
                        if subsubelem.tag == "xmin":
                            xmin1 = str(i)
                        if subsubelem.tag == "xmax":
                            xmax1 = str(i)
                        if subsubelem.tag == "ymin":
                            ymin1 = str(i)
                        if subsubelem.tag == "ymax":
                            ymax1 = str(i)

#                         fn = [x for x in os.listdir('./output_with_bgs/')]
        g = cv2.imread('./plate_with_bgs/'+xml_file.split('.')[0]+'.jpg')

        strt = (int(xmin1),int(ymin1))
        stp = (int(xmax1),int(ymax1))
        gg = cv2.rectangle(g,strt,stp,(0,255,0),2)
        cv2.imshow('imgg',gg)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [11]:
csv_name = 'train_csv//Plates'

if os.path.exists(csv_name+'.csv'):
    df = pd.read_csv(csv_name+'.csv')
    df.columns =['image_id','x','y','w','h','classes']
    df['w'] = df['w'] - df['x']
    df['h'] = df['h'] - df['y']
    df['x_center'] = df['x'] + df['w']/2
    df['y_center'] = df['y'] + df['h']/2
    df['classes'] = 0
    from tqdm.auto import tqdm
    import shutil as sh
    df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]

In [12]:
df.head()

,image_id,x,y,w,h,x_center,y_center,classes
0,OEF063_bg,40,32,110,54,95.0,59.0,0
1,V195KH1_bg,107,111,61,31,137.5,126.5,0
2,6W79TZ1_bg,18,171,76,30,56.0,186.0,0
3,9832GX_bg,43,50,70,34,78.0,67.0,0
4,90N9TI_bg,44,142,84,41,86.0,162.5,0


In [13]:
index = list(set(df.image_id))
len(index)

43500

In [14]:
# image_resize = 320
source = 'plate_with_bgs'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/image_resize
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))